# Notes on PyOpenCL mainly.
## See original notes by Derek Mendez here: 
http://prickly-pythons.github.io/python_code/pricklies_speedops.html

In [31]:
import pyopencl
import numpy as np
import time as time

## Method # 1: pure numpy

In [32]:
# Define a method that will 
# calculate I(q) where q is a vector 
# and I is a complex number:
def method1(q_vecs, atom_vecs):
    Nq = q_vecs.shape[0]
    ampsR = np.zeros( Nq ) 
    ampsI = np.zeros( Nq )
    for i_q, q in enumerate( q_vecs):
        qx,qy,qz = q
        for i_atom, atom in enumerate( atom_vecs):
            ax,ay,az = atom
            phase = qx*ax + qy*ay + qz*az
            ampsR[i_q] += np.cos( -phase)
            ampsI[i_q] += np.sin( -phase)
    I = ampsR**2 + ampsI**2 
    return I

In [33]:
# Set up random set of q (qx,qy,qz) vectors and atom_vecs vectors
q_vecs = np.random.random((10000, 3))
# Pick a random set of ax,ay,az (100 atom vectors)
atom_vecs = np.random.random((100, 3))

In [34]:
# Timing of this method:
t1 = time.time()
I = method1(q_vecs, atom_vecs)
t2 = time.time()
I[0:10]

array([8712.12100683, 9619.37310958, 8437.28900954, 9284.73212745,
       9046.97141614, 9559.18943437, 9876.47812006, 9052.87485658,
       8254.13471571, 9499.61685364])

In [35]:
print('Method 1 took %.4s seconds' % (t2-t1))

Method 1 took 4.56 seconds


OBS: You can also do profiling of method1 by creating a separate file, see profiling_method1.py.
I cloned and installed the profiler from:  https://github.com/rkern/line_profiler. Also, I changed the first line of kernprof.py to  !/Users/Karen/anaconda2/bin/python2. 

Run profiling with:
```
kernprof -l profiling_method1.py
py2 -m line_profiler profiling_method1.py.lprof
```
Which gives me:

## Method # 2: Embarrassingly parallel
Using multiple processors to speed up method 1

In [36]:
from joblib import Parallel, delayed
def method2(q_vecs, atom_vecs, my_method, n_jobs=4,):
    q_vecs_split = np.array_split(q_vecs, n_jobs)
    results = Parallel(n_jobs=n_jobs)(
        delayed(my_method)(qs,atom_vecs) 
        for qs in q_vecs_split)
    return np.concatenate(results,0) 

In [37]:
# Timing of this method:
t1 = time.time()
I = method2(q_vecs, atom_vecs, my_method=method1, n_jobs=4)
t2 = time.time()

In [38]:
print('Method 2 took %.4s seconds' % (t2-t1))

Method 2 took 1.27 seconds


## Method # 3: pure numpy
First, let's check configuration of numpy: I installed openblas, and re-installed numpy directly from github with a configuration to use OpenBlas, as described here: 
[https://dedupe.io/developers/library/en/latest/OSX-Install-Notes.html]()

In [39]:
np.show_config()

lapack_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/opt/openblas/lib']
    define_macros = [('HAVE_CBLAS', None)]
    language = c
blas_opt_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/opt/openblas/lib']
    define_macros = [('HAVE_CBLAS', None)]
    language = c
openblas_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/opt/openblas/lib']
    define_macros = [('HAVE_CBLAS', None)]
    language = c
blis_info:
  NOT AVAILABLE
openblas_lapack_info:
    libraries = ['openblas', 'openblas']
    library_dirs = ['/usr/local/opt/openblas/lib']
    define_macros = [('HAVE_CBLAS', None)]
    language = c
lapack_mkl_info:
  NOT AVAILABLE
blas_mkl_info:
  NOT AVAILABLE


Using just numpy is SUPER fast, but memory intensive because it stores the phase terms as opposed to computing them on the fly. Let's time it:

In [40]:
def method3(q_vecs, atom_vecs):
    phases = np.dot( q_vecs, atom_vecs.T) 
    # T is the transpose, hence (Nq x 3) dotted with (3 x Natom)
    # This results in an (Nq x Natom) array
    
    #In the end we want Nq amplitudes (both real and imaginary)
    cosph = np.cos( -phases) 
    sinph = np.sin( -phases)
    ampsR = np.sum( cosph, axis=1) # summing over the atoms axis... 
    ampsI = np.sum( sinph, axis=1)

    # Take the mod-squared and return 
    I = ampsR**2 + ampsI**2 
    return I
# Timing this method:
t1 = time.time()
I = method3(q_vecs, atom_vecs)
t2 = time.time()
print('Method 3 took %.4s seconds' % (t2-t1))

Method 3 took 0.04 seconds


## Method # 4: numexpr
Introducing multi-threading to speed-up element-wise numpy operations!<br> 
Can somtimes make numpy run faster, but not really in this case:

In [41]:
import numexpr as ne

In [42]:
def method4(q_vecs, atom_vecs):
    phases = np.dot( q_vecs, atom_vecs.T) # T is the transpose, hence (Nq x 3) dotted with (3 x Natom)
    # This results in an (Nq x Natom) array
    
    # In the end we want Nq amplitudes (both real and imaginary)
    cosph = ne.evaluate('cos( -phases)')
    sinph = ne.evaluate('sin( -phases)')
    ampsR = np.sum( cosph, axis=1) # summing over the atoms axis... 
    ampsI = np.sum( sinph, axis=1)

    # Take the mod-squared and return 
    #I = ne.evaluate('ampsR**2 + ampsI**2')
    I = ampsR**2 + ampsI**2 
    return I
# Timing this method:
t1 = time.time()
I = method4(q_vecs, atom_vecs)
t2 = time.time()
print('Method 4 took %.4s seconds' % (t2-t1))

Method 4 took 0.01 seconds


## Method # 5: OpenCL
Let's start by looking up the platform and GPU(s) of this computer, using pyopencl:

In [43]:
import pyopencl as cl
def get_context_queue():
#   list the platforms
    platforms = cl.get_platforms()
    print("Found platforms (will use first listed):", platforms)
#   select the gpu
    my_gpu = platforms[0].get_devices(
        device_type=cl.device_type.GPU)
    assert( my_gpu)
    print("Found GPU(s):", my_gpu)
#   create the context for the gpu, and the corresponding queue
    context = cl.Context(devices=my_gpu)
    queue = cl.CommandQueue(context)
    return context, queue

In [44]:
context,queue = get_context_queue()

('Found platforms (will use first listed):', [<pyopencl.Platform 'Apple' at 0x7fff0000>])
('Found GPU(s):', [<pyopencl.Device 'Iris Pro' on 'Apple' at 0x1024500>])


In [45]:
import pyopencl.array as clarray
def method5( q_vecs, atom_vecs, context, queue):
    Nato = atom_vecs.shape[0]
    Nq = q_vecs.shape[0]
    # these are the output host buffers which will also be transferred to the device, updated on the device and then copied back to the host.
    ampsR = np.ascontiguousarray(np.zeros( Nq, dtype=np.float32) )
    ampsI = np.ascontiguousarray(np.zeros( Nq, dtype=np.float32) )
    # OpenCL C source code , this is the instructions to the GPU workers
    # note this code is essentially the inner-most loop of method 1, such that
    # each worker is doing a single iteration of the outer-loop of method 1.
    # this allows for insane speedups, without the need to use excess memory
    # as in the numpy case (although GPUs have limited memory)
    kernel = """ 
    __kernel void sim_amps(__global float* q_vecs,
                            __global float* atom_vecs,
                             __global float* ampsR,
                             __global float* ampsI,
                             int Nq, int Natoms){
    //  this is the unique ID of each worker, and each worker will be loading a single q vec
        int g_i = get_global_id(0);
        float qx,qy,qz,ax,ay,az, cph, sph, phase;
    //  we pass 1D arrays to openCL, in row-major order
        qx = q_vecs[g_i*3];
        qy = q_vecs[g_i*3+1];
        qz = q_vecs[g_i*3+2];
        for(int i =0; i < Natoms; i++){
            ax = atom_vecs[i*3];
            ay = atom_vecs[i*3+1];
            az = atom_vecs[i*3+2];
            phase = ax*qx + ay*qy + az*qz;
            cph = native_cos(phase); // native openCL trig functions
            sph = native_sin(phase);
            ampsR[g_i] += cph;
            ampsI[g_i] += sph;
            
        }
    }
    """
    #   setup opencl, compile bugs will show up here
    program = cl.Program(context, kernel).build()

#   move host arrays to GPU device, note forcing q_vecs and atom_vecs to be contiguous , ampsR and ampsI are already contiguous
    qs_dev = clarray.to_device(queue, np.ascontiguousarray(q_vecs.astype(np.float32)))
    atoms_dev = clarray.to_device(queue, np.ascontiguousarray(atom_vecs.astype(np.float32)))
    ampsR_dev = clarray.to_device(queue, ampsR)
    ampsI_dev = clarray.to_device(queue, ampsI)

#   specify scalar args (just tell openCL which kernel args are scalar)
    program.sim_amps.set_scalar_arg_dtypes(
            [None, None, None,None,np.int32, np.int32])
#   run the kernel
#   note there are 3 pre-arguments to our kernel, these are the queue, 
#   the total number of workers, and the desired worker-group size. 
#   Leaving worker-group size as None lets openCL decide a value (I think)
    program.sim_amps(queue, (Nq,), None, qs_dev.data, atoms_dev.data, 
        ampsR_dev.data, ampsI_dev.data, np.int32(Nq), np.int32(Nato))

#   transfer data from device back to host
#    you can try to optimize enqueue_copy by passing different flags 
    cl.enqueue_copy(queue, ampsR, ampsR_dev.data)
    cl.enqueue_copy(queue, ampsI, ampsI_dev.data)
    
    I = ampsR**2 + ampsI**2
    return I

Increasing the number of q and atom vectors,<br>
we will se a big improvement in time:

In [46]:
q_vecs = np.random.random((100000, 3))
atom_vecs = np.random.random((1000, 3))

In [47]:
t1 = time.time()
I = method4(q_vecs, atom_vecs)
t2 = time.time()
print('Method 4 took %.4s seconds' % (t2-t1))
I[0:10]

Method 4 took 1.30 seconds


array([980281.77204445, 953003.89850894, 932079.88938346, 931743.56519526,
       966050.92340825, 960212.25987076, 848684.89946519, 868769.74929962,
       887698.05051446, 988611.67544165])

In [48]:
t1 = time.time()
I = method5(q_vecs, atom_vecs, context, queue)
t2 = time.time()
print('Method 5 took %.4s seconds' % (t2-t1))
I[0:10]

Method 5 took 0.03 seconds


array([980320.7 , 953040.9 , 932117.3 , 931782.4 , 966091.9 , 960250.1 ,
       848720.5 , 868805.4 , 887733.75, 988650.5 ], dtype=float32)